## Imports

In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re

## Import data from PDF

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

## Geocoder Setup and Execution

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (360 Ocean Dr #401s, Key Biscayne, FL 33149, U...
2     (20155 NE 38th Ct APT 2704, Aventura, FL 33180...
3     (17141 Collins Ave #1201, Sunny Isles Beach, F...
4     (6103 Aqua Ave APT 1004, Miami Beach, FL 33141...
5     (9521 E Bay Harbor Dr #304, Bay Harbor Islands...
6     (19111 Collins Ave APT 2201, Sunny Isles Beach...
7     (3301 NE 183rd St UNIT 2509, Aventura, FL 3316...
8     (851 NE 1st Ave #3112, Miami, FL 33132, USA, (...
9     (851 NE 1st Ave #1901, Miami, FL 33132, USA, (...
10    (2900 NE 7th Ave Unit 507, Miami, FL 33137, US...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Oceana Key Biscayne\r360 Ocean Dr 401S\rKey Bi...,4/18/2022,6/22/2022,44,"$9,000,000.00",3791,"$2,374.04",David Siddons,Douglas Elliman,Juan De Lapuerta,Emerald Real Estate Inc,Oceana Key Biscayne 360 Ocean Dr 401S Key Bisc...,Oceana Key Biscayne,360 Ocean Dr 401S Key Biscayne,"(360 Ocean Dr #401s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0
2,Porto Vita\r20155 NE 38th Ct 2704\rAventura,11/24/2020,6/20/2022,405,"$4,900,000.00",5792,$845.99,Jill Hertzberg,Coldwell Banker Realty,Roberta Esquenazi,Marina Realty Group,Porto Vita 20155 NE 38th Ct 2704 Aventura,Porto Vita,20155 NE 38th Ct 2704 Aventura,"(20155 NE 38th Ct APT 2704, Aventura, FL 33180...","(25.9625239, -80.1257694, 0.0)",25.962524,-80.125769,0.0
3,Muse\r17141 Collins Ave 1201\rSunny Isles Beach,3/6/2022,6/21/2022,75,"$4,750,000.00",3195,"$1,486.70",Sandra Pariente,Charles Rutenberg Realty,Neal Klotsman,Douglas Elliman,Muse 17141 Collins Ave 1201 Sunny Isles Beach,Muse,17141 Collins Ave 1201 Sunny Isles Beach,"(17141 Collins Ave #1201, Sunny Isles Beach, F...","(25.934944, -80.12091819999999, 0.0)",25.934944,-80.120918,0.0
4,Spear at Aqua Condo\r6103 Aqua Ave 1004\rMiami...,2/3/2022,6/21/2022,68,"$2,400,000.00",2203,"$1,089.42",Jeffrey Miller,One Sotheby's International,Jason Angrist,"Compass Florida, Inc",Spear at Aqua Condo 6103 Aqua Ave 1004 Miami B...,Spear at Aqua Condo,6103 Aqua Ave 1004 Miami Beach,"(6103 Aqua Ave APT 1004, Miami Beach, FL 33141...","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0
5,Bijou Bay Harbor\r9521 E Bay Harbor Dr 304\rBa...,10/1/2021,6/22/2022,234,"$2,300,000.00",2082,"$1,104.71",Patrick Vysata,Luxe Living Realty,NON-MLS MEMBER,Mar Non-MLS Member,Bijou Bay Harbor 9521 E Bay Harbor Dr 304 Bay ...,Bijou Bay Harbor,9521 E Bay Harbor Dr 304 Bay Harbor Islands,"(9521 E Bay Harbor Dr #304, Bay Harbor Islands...","(25.886001, -80.1287188, 0.0)",25.886001,-80.128719,0.0
6,Ocean Two\r19111 Collins Ave 2201\rSunny Isles...,12/1/2021,6/24/2022,113,"$2,150,000.00",2665,$806.75,Wendy Cohen,One Sotheby's International,Nir-David Perets,The Corcoran Group,Ocean Two 19111 Collins Ave 2201 Sunny Isles B...,Ocean Two,19111 Collins Ave 2201 Sunny Isles Beach,"(19111 Collins Ave APT 2201, Sunny Isles Beach...","(25.9534131, -80.1197571, 0.0)",25.953413,-80.119757,0.0
7,The Peninsula II Condo\r3301 NE 183 St 2509\rA...,5/3/2022,6/21/2022,10,"$1,715,000.00",3073,$558.09,Katherine Jattin,Avanti Way Realty LLC,Perla Bursztein,"Compass Florida, Inc",The Peninsula II Condo 3301 NE 183 St 2509 Ave...,The Peninsula II Condo,3301 NE 183 St 2509 Aventura,"(3301 NE 183rd St UNIT 2509, Aventura, FL 3316...","(25.9462602, -80.1364097, 0.0)",25.946260,-80.136410,0.0
8,Paramount Worldcenter\r851 NE 1st Ave 3112\rMiami,11/19/2019,6/22/2022,900,"$1,675,000.00",2611,$721.98,Kayce Driscoll,Douglas Elliman,Sildy Cervera PA,Cervera Real Estate Inc,Paramount Worldcenter 851 NE 1st Ave 3112 Miami,Paramount Worldcenter,851 NE 1st Ave 3112 Miami,"(851 NE 1st Ave #3112, Miami, FL 33132, USA, (...","(25.782502, -80.1917783, 0.0)",25.782502,-80.191778,0.0
9,Paramount Worldcenter\r851 NE 1st Ave 1901\rMiami,11/17/2021,6/21/2022,212,"$1,550,000.00",2204,$703.27,Irina Vasserman,Atlantic VIP Realty Group LLC,Jessica Turner,Blackbook Properties,Paramount Worldcenter 851 NE 1st Ave 1901 Miami,Paramount Worldcenter,851 NE 1st Ave 1901 Miami,"(851 NE 1st Ave #1901, Miami, FL 33132, USA, (...","(25.782502, -80.1917783, 0.0)",25.782502,-80.191778,0.0
10,B0iscayne Beach Condo\r2900 NE 7th Ave 507\rMiami,4/19/2022,6/22/2022,64,"$1,500,000.00",1604,$935.16,David Concheso,"Related ISG Realty, LLC",Sumita Sengupta,Douglas Elliman,B0iscayne Beach Condo 2900 NE 7th Ave 507 Miami,B,0iscayne Beach Condo 2900 NE 7th Ave 507 Miami,"(2900 NE 7th Ave Un

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [16]:
df.at[10,'building_name']=('Biscayne Beach Condo')
df.at[10,'address']=('2900 NE 7th Ave 507 Miami')

In [17]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('June 19th - June 25th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [18]:
m.save('index.html')